1. Load the height/weight data from the file heightWeightData.txt. The first column is the class label (1=male, 2=female), the second column is height, the third weight. Start by replacing the weight column by the product of height and weight.

For the Fisher’s linear discriminant analysis as discussed in the class, send the python/matlab code and answers for the following questions:

a. What’s the SB matrix?

b. What’s the SW matrix?

c. What’s the optimal 1d projection direction?

d. Project the data in the optimal 1d projection direction. Set the decision threshold as the middle point between the 
projected means. What’s the misclassification error rate?

e. What’s your height and weight? What’s the model prediction for your case (male/female)?

In [1]:
#Imports
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(suppress=True)

In [20]:
data = np.genfromtxt("heightWeightData.txt", delimiter=",")

#Weight is 3rd Column
np.set_printoptions(suppress=True)
new_data = np.zeros(data.shape)
for i in range(int(new_data.shape[0])):
    new_data[i, 0] = data[i, 0]
    new_data[i, 1] = data[i, 1]
    new_data[i, 2] = np.multiply(data[i, 1], data[i, 2])

In [3]:
#Implementing Fisher's Linear Discriminant Analysis
#Let's group data first
#Count males (=1) and females (=2)
nr_males = 0
nr_females = 0
for i in range(int(new_data.shape[0])):
    if new_data[i, 0] == 1:
        nr_males+=1
    elif new_data[i, 0] == 2:
        nr_females+=1
#print(nr_males, nr_females)
#Concatenate Class Sizes
class_sizes = np.array([nr_males, nr_females])

#Assign Classes
males = np.zeros([nr_males, new_data.shape[1]])
females = np.zeros([nr_females, new_data.shape[1]])
m_index = 0
f_index = 0
for index in range(int(new_data.shape[0])):
    if new_data[index, 0] == 1:
        males[m_index] = new_data[index]
        m_index+=1
    elif new_data[index, 0] == 2:
        females[f_index] = new_data[index]
        f_index+=1

#Calculate means vector for each class
#Drop Label Column
f_males = males[:, 1:]
f_females = females[:, 1:]
#Calculate mean vector for each class
mean_males = np.mean(a=f_males, axis=0)
mean_females = np.mean(a=f_females, axis=0)

print("Mean Vector for Males Class: \n", mean_males,"\nMean Vector for Females Class: \n", mean_females)

#Calculate Overall Mean
overall_mean = np.mean(new_data[:, 1:], axis=0)
#print(overall_mean)

#Let's Compute Between Class Scatter Matrix S_B
#Concatenate Mean Vectors
class_means = np.array([mean_males, mean_females])
#Iterate through each class
n_S_B = np.zeros((2,2))
S_B = np.zeros((2,2))
for i in range(2):
    n_samples = class_sizes[i]
    mean_of_class = class_means[i].reshape(2,1) #Reshape Mean Vectors into Column Vectors to perform Computation
    ovrl_mean = overall_mean.reshape(2,1) #Reshape Overall Mean Vector into Column Vector to perform Computation
    #Taking into account sample sizes
    n_S_B += n_samples * (mean_of_class - ovrl_mean).dot((mean_of_class - ovrl_mean).T)
    #n_S_B += n_samples * np.outer((mean_of_class - ovrl_mean), (mean_of_class - ovrl_mean))
    #Without sample sizes
    S_B += (mean_of_class - ovrl_mean).dot((mean_of_class - ovrl_mean).T)
    #S_B += np.outer((mean_of_class - ovrl_mean), (mean_of_class - ovrl_mean))

#S_B Matrix
print('Between-Class Scatter Matrix, taking into account sample sizes:\n', n_S_B)
print('Between-Class Scatter Matrix, not regarding sample sizes:\n', S_B)


#Let's Compute Within Class Scatter Matrix S_W
#Calculate the scatter matrices for the SW (Scatter within) and sum the elements up
#First reshape vectors to perform computations
#Scatter Matrix for Males Class
scatter_males = np.zeros((2, 2))
for row in f_males:
    row, mv = row.reshape(2,1), mean_males.reshape(2,1) # make column vectors
    scatter_males += (row-mv).dot((row-mv).T)

#Scatter Matrix for Females Class
scatter_females = np.zeros((2, 2))
for row in f_females:
    row, mv = row.reshape(2,1), mean_females.reshape(2,1) #make column vectors
    scatter_females += (row-mv).dot((row-mv).T)

#Scatter Within Matrix is the sum of both of the Scatter Matrix above
S_W = scatter_males + scatter_females # sum class scatter matrices
print("Within Class Scatter Matrix is:\n", S_W)

Mean Vector for Males Class: 
 [  182.01013699 14552.85501781] 
Mean Vector for Females Class: 
 [ 165.28540146 9757.31728073]
Between-Class Scatter Matrix, taking into account sample sizes:
 [[1.33211786e+04 3.81962480e+06]
 [3.81962480e+06 1.09521342e+09]]
Between-Class Scatter Matrix, not regarding sample sizes:
 [[     152.84841103    43826.72132582]
 [   43826.72132582 12566578.14875751]]
Within Class Scatter Matrix is:
 [[1.34819629e+04 2.38635072e+06]
 [2.38635072e+06 1.07660688e+09]]


In [18]:
#Obtain the optimal 1D Projection Direction
#Using S_B that takes into account the number of samples n_S_B
#eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(n_S_B))

#Using S_B that takes into account the number of samples n_S_B
eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))

for i in range(len(eig_vals)):
    eigvec_sc = eig_vecs[:,i].reshape(2,1)   
    print('\nEigenvector {}: \n{}'.format(i+1, eigvec_sc.real))
    print('Eigenvalue {:}: {:.2e}'.format(i+1, eig_vals[i].real))
    
#Check the eigenvector-eigenvalue calculation
for i in range(len(eig_vals)):
    eigv = eig_vecs[:,i].reshape(2,1)
    np.testing.assert_array_almost_equal(np.linalg.inv(S_W).dot(S_B).dot(eigv),
                                         eig_vals[i] * eigv,
                                         decimal=6, err_msg='', verbose=True)
print('\nEverything was correctly calculated!')


Eigenvector 1: 
[[-0.99999392]
 [ 0.00348754]]
Eigenvalue 1: 1.73e-18

Eigenvector 2: 
[[-0.99999289]
 [-0.00377042]]
Eigenvalue 2: 1.42e-02

Everything was correctly calculated!


In [29]:
# Make a list of (eigenvalue, eigenvector) tuples
#eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
#eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)

# Visually confirm that the list is correctly sorted by decreasing eigenvalues

#print('Eigenvalues in decreasing order:\n')
#for i in eig_pairs:
    #print(i[0])

# 4. Compute the Eigenvalues and Eigenvectors of SW^-1 SB
eigval, eigvec = np.linalg.eig(np.dot(np.linalg.inv(S_W),S_B))
    
# 5. Select the two largest eigenvalues 
eigen_pairs = [[np.abs(eigval[i]),eigvec[:,i]] for i in range(len(eigval))]
eigen_pairs = sorted(eigen_pairs,key=lambda k: k[0],reverse=True)
w = np.hstack((eigen_pairs[0][1][:,np.newaxis].real,eigen_pairs[1][1][:,np.newaxis].real)) # Select two largest
#Matrix W
print("Matrix W is: \n", w)

Matrix W is: 
 [[-0.99999289 -0.99999392]
 [-0.00377042  0.00348754]]


In [31]:
#Transforming the samples onto the new subspace
#Project the means and take their mean; We multiply w by -1, i.e. -w, to get positive threshold!
tot = 0
#for mean in class_means:
tot = np.dot(w, class_means)
    #print(tot)
w0 = 0.5 * tot
print("Calculated threshold is: ", w0)

Calculated threshold is:  [[  -173.64661976 -12155.00475843]
 [    -0.05490772    -10.42068166]]


In [27]:
#Calculate Error
#For each input project the point
features = data[:, 1:]
labels = data[:,0]
projected = np.zeros((int(features.shape[0])))
for i in range(int(features.shape[0])):
    projected[i] = np.dot(-w, features[i].T).T
projected

ValueError: setting an array element with a sequence.

In [16]:
#Assign Predictions
predictions = []
for item in projected:
    if item >= w0:
        predictions.append(1)
    else:
        predictions.append(2)
#predictions

In [17]:
#Check Classification
errors = (labels != predictions)
n_errors = sum(errors)

error_rate = (n_errors/len(predictions) * 100)
print("Error Rate is: ", error_rate, "%")

Error Rate is:  34.76190476190476 %


In [10]:
#My case
my_height = 164
my_weight = 65
my_features = np.array([my_height, my_weight*my_height])
my_ground_truth = "Male"

#My Prediction
my_projection = np.dot(w, my_features.T).T
if my_projection >= w0:
    my_pred = "Male"
else:
    my_pred = "Female"

print("In my case I was predicted as: ", my_pred, " which is ", my_ground_truth==my_pred)

In my case I was predicted as:  Female  which is  False


https://sebastianraschka.com/Articles/2014_python_lda.html

http://goelhardik.github.io/2016/10/04/fishers-lda/

https://www.python-course.eu/linear_discriminant_analysis.php

https://github.com/goelhardik/projects/blob/master/fishers_lda/lda.py

https://scikit-learn.org/stable/modules/lda_qda.html

In [11]:
#Using sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
clf = LinearDiscriminantAnalysis()
clf.fit(features, labels)
LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='eigen', store_covariance=False, tol=0.0001)
print(clf.get_params())
print(clf.predict(features))
sum(labels!=clf.predict(features))


{'n_components': None, 'priors': None, 'shrinkage': None, 'solver': 'svd', 'store_covariance': False, 'tol': 0.0001}
[2. 2. 2. 2. 2. 2. 1. 2. 1. 2. 2. 1. 1. 2. 2. 2. 2. 1. 2. 1. 2. 2. 2. 1.
 1. 2. 2. 2. 2. 2. 1. 2. 2. 2. 1. 2. 2. 2. 1. 2. 2. 1. 2. 2. 1. 1. 2. 1.
 2. 2. 1. 2. 2. 1. 2. 1. 1. 2. 2. 2. 1. 2. 1. 2. 2. 2. 2. 1. 1. 2. 2. 1.
 2. 1. 2. 2. 1. 2. 1. 2. 2. 2. 2. 2. 2. 1. 2. 1. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 1. 1. 1. 1. 1. 2. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 2.
 2. 2. 2. 2. 2. 1. 1. 1. 2. 1. 1. 1. 1. 2. 2. 2. 1. 2. 2. 2. 2. 1. 2. 2.
 1. 2. 2. 2. 2. 2. 1. 2. 1. 2. 2. 1. 2. 2. 1. 2. 1. 2. 2. 2. 2. 2. 2. 1.
 2. 1. 2. 2. 2. 1. 2. 2. 2. 2. 2. 2. 2. 1. 2. 2. 1. 2. 2. 2. 1. 2. 2. 1.
 2. 2. 2. 2. 1. 2. 2. 1. 2. 2. 1. 2. 1. 1. 1. 1. 2. 1.]


25

In [225]:
error = (25/210)*100
error

11.904761904761903

In [21]:
overall_mean

array([  171.0992381 , 11424.33754171])